In [15]:
import pandas as pd
import numpy as np
import os
from run_fall_detection_model  import run_fall_detection_model


In [16]:

def search_csv_files(directory,activities_of_interest = None):
    csv_files = []
    for current_folder, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                if activities_of_interest is not None:
                    for activity in activities_of_interest:
                        if activity in file:
                            full_path = os.path.join(current_folder, file)
                            csv_files.append(full_path)
                            break
                else:
                    full_path = os.path.join(current_folder, file)
                    csv_files.append(full_path)
                    break
    return csv_files




falls_files = search_csv_files(directory='output', activities_of_interest=['Activity13', 'Activity14', 'Activity15'])
adl_files = search_csv_files(directory='output', activities_of_interest=['Activity1', 'Activity2', 'Activity3', 'Activity4', 'Activity5', 'Activity6', 'Activity7', 'Activity8', 'Activity9', 'Activity10', 'Activity11', 'Activity12'])

len(falls_files), len(adl_files)


(198, 678)

In [17]:
def load_and_pad(file, use_gyro=False, max_len=300):
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Define the columns to use
    if use_gyro:
        cols = ['Accelerometer: x-axis (g)', 'Accelerometer: y-axis (g)', 'Accelerometer: z-axis (g)',
                'Gyroscope: x-axis (rad/s)', 'Gyroscope: y-axis (rad/s)', 'Gyroscope: z-axis (rad/s)']
    else:
        cols = ['Accelerometer: x-axis (g)', 'Accelerometer: y-axis (g)', 'Accelerometer: z-axis (g)']
    
    # Clean and convert data for each column
    for col in cols:
        if df[col].dtype == 'object':  # If column has string values
            # Replace incorrect number format (assuming European format with '.' as thousand separator)
            df[col] = df[col].astype(str).str.replace('.', '', n=1)  # Remove first period (thousand separator)
            df[col] = pd.to_numeric(df[col].str.replace(',', '.'), errors='coerce')  # Replace comma with period for decimal

    # Extract features and convert to numpy array
    features = df[cols].to_numpy().astype('float32')
    
    # Pad or truncate as needed
    if features.shape[0] > max_len:
        features = features[:max_len, :]
    elif features.shape[0] < max_len:
        pad = np.zeros((max_len - features.shape[0], features.shape[1]), dtype='float32')
        features = np.vstack((features, pad))
    
    return features




def build_dataset(falls_files, adl_files, max_len=300):
    X_acc, X_all, y = [], [], []

    for file in falls_files:
        X_acc.append(load_and_pad(file, use_gyro=False, max_len=max_len))
        X_all.append(load_and_pad(file, use_gyro=True, max_len=max_len))
        y.append(1)

    for file in adl_files:
        X_acc.append(load_and_pad(file, use_gyro=False, max_len=max_len))
        X_all.append(load_and_pad(file, use_gyro=True, max_len=max_len))
        y.append(0)

    return np.array(X_acc), np.array(X_all), np.array(y)



In [18]:
X_acc, X_all, y = build_dataset(falls_files, adl_files)


print("Accelerometer only:", X_acc.shape)
print("Accelerometer + Gyroscope:", X_all.shape)
print("Labels:", y.shape)

Accelerometer only: (876, 300, 3)
Accelerometer + Gyroscope: (876, 300, 6)
Labels: (876,)


In [19]:
from imblearn.over_sampling import SMOTE

USE_GYRO = False

X = X_all if USE_GYRO else X_acc
X_2d = X.reshape((X.shape[0], -1))

# Remove samples with NaNs
mask = ~np.isnan(X_2d).any(axis=1)
X_clean = X_2d[mask]
y_clean = y[mask]

print(f"Removed {len(y) - len(y_clean)} samples due to NaNs.")

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_clean, y_clean)

# Reshape back to 3D for CNN
X_resampled = X_resampled.reshape((-1, 300, X.shape[2]))

print("✅ SMOTE done:")
print("X shape:", X_resampled.shape)
print("Label counts:", np.bincount(y_resampled))

Removed 19 samples due to NaNs.
✅ SMOTE done:
X shape: (1318, 300, 3)
Label counts: [659 659]


/Users/amir/Documents/projects/fall-detection/.venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [20]:
model, history = run_fall_detection_model(X_acc, X_all, y)

Removed 19 samples due to NaNs.
✅ SMOTE applied
X shape: (1318, 300, 3)
Label counts: [659 659]
Epoch 1/20


/Users/amir/Documents/projects/fall-detection/.venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7132 - loss: 0.5496 - val_accuracy: 0.7962 - val_loss: 0.4755
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8288 - loss: 0.4486 - val_accuracy: 0.8009 - val_loss: 0.4702
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8399 - loss: 0.4074 - val_accuracy: 0.7962 - val_loss: 0.4679
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8492 - loss: 0.3733 - val_accuracy: 0.7915 - val_loss: 0.4704
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8511 - loss: 0.3454 - val_accuracy: 0.7867 - val_loss: 0.4650
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8453 - loss: 0.3741 - val_accuracy: 0.7915 - val_loss: 0.4885
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8368 - loss: 0.3833 - val_accuracy: 0.7915 - val_loss: 0.5009
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8557 - loss: 0.3467 - val_accuracy: 0.8104 - val_loss: 0.4938
Epo

In [21]:
model, history = run_fall_detection_model(X_acc, X_all, y, use_gyro=True)

Removed 19 samples due to NaNs.
✅ SMOTE applied
X shape: (1318, 300, 6)
Label counts: [659 659]
Epoch 1/20


/Users/amir/Documents/projects/fall-detection/.venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6641 - loss: 0.6046 - val_accuracy: 0.7962 - val_loss: 0.4581
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8428 - loss: 0.4252 - val_accuracy: 0.8009 - val_loss: 0.4318
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8386 - loss: 0.3733 - val_accuracy: 0.8009 - val_loss: 0.4497
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8425 - loss: 0.3852 - val_accuracy: 0.8057 - val_loss: 0.4575
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8489 - loss: 0.3404 - val_accuracy: 0.7962 - val_loss: 0.4684
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8763 - loss: 0.3201 - val_accuracy: 0.8009 - val_loss: 0.4663
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8515 - loss: 0.3214 - val_accuracy: 0.7867 - val_loss: 0.5393
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8505 - loss: 0.3226 - val_accuracy: 0.8057 - val_loss: 0.5646
Epo

In [22]:
# Without SMOTE

model, history = run_fall_detection_model(X_acc, X_all, y, apply_smote=False)

Removed 19 samples due to NaNs.
Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6979 - loss: 0.5168 - val_accuracy: 0.7664 - val_loss: 0.3903
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7954 - loss: 0.3555 - val_accuracy: 0.7737 - val_loss: 0.3887
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7741 - loss: 0.3479 - val_accuracy: 0.7226 - val_loss: 0.4000
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8085 - loss: 0.3323 - val_accuracy: 0.6861 - val_loss: 0.4136
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7985 - loss: 0.3391 - val_accuracy: 0.7226 - val_loss: 0.4040
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7925 - loss: 0.3261 - val_accuracy: 0.7299 - val_loss: 0.4277
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8100 - loss: 0.3214 - val_accuracy: 0.7153 - val_loss: 0.4269
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8059 - loss: 0.3457 - 